# `llama.cpp`

More info about using `llama.cpp` models with outlines [here](https://dottxt-ai.github.io/outlines/latest/reference/models/llama_cpp/)

Table of Contents:
- [JSON Generation](#json-generation)
- [Choice Generation](#choice-generation)
- [Text Generation](#text-generation)


## JSON Generation

Example based on https://dottxt-ai.github.io/outlines/latest/cookbook/extraction/

In [1]:
from enum import Enum

import jinja2
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from pydantic import BaseModel

from outlines_haystack.generators.llama_cpp import LlamaCppJSONGenerator

In [2]:
class Pizza(str, Enum):
    margherita = "Margherita"
    calzone = "Calzone"


class Order(BaseModel):
    pizza: Pizza
    number: int

In [3]:
prompt_template = """You are the owner of a pizza parlor. Customers \
send you orders from which you need to extract:

1. The pizza that is ordered
2. The number of pizzas

# EXAMPLE

ORDER: I would like one Margherita pizza
RESULT: {"pizza": "Margherita", "number": 1}

# OUTPUT INSTRUCTIONS

Answer in valid JSON. Here are the different objects relevant for the output:

Order:
    pizza (str): name of the pizza
    number (int): number of pizzas

Return a valid JSON of type "Order"

# OUTPUT

ORDER: {{ order }}
RESULT: """

In [4]:
generator = LlamaCppJSONGenerator(
    repo_id="microsoft/Phi-3-mini-4k-instruct-gguf",
    file_name="Phi-3-mini-4k-instruct-q4.gguf",
    schema_object=Order,
)

### Standalone

In [5]:
generator.warm_up()

/Users/edoardoabati/projects/opensource/outlines-haystack/.venv/lib/python3.12/site-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `hf_hub_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
llama_model_load_from_file_impl: using device Metal (Apple M1 Pro) - 25558 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from /Users/edoardoabati/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct-gguf/snapshots/a64113399c2f6b8ad3e11c394733a2ddadaa7f33/./Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv  

In [6]:
# we use Jinja2 to render the template
prompt = jinja2.Template(prompt_template).render(order="Is it possible to have 12 margheritas?")
generator.run(prompt=prompt, generation_kwargs={"temperature": 0.5})

llama_perf_context_print:        load time =    1988.72 ms
llama_perf_context_print: prompt eval time =    1988.47 ms /   173 tokens (   11.49 ms per token,    87.00 tokens per second)
llama_perf_context_print:        eval time =     541.04 ms /    15 runs   (   36.07 ms per token,    27.72 tokens per second)
llama_perf_context_print:       total time =    2912.94 ms /   188 tokens
llama_perf_context_print:    graphs reused =         14


{'structured_replies': ['{"pizza": "Margherita", "number": 12']}

### In a Pipeline

In [7]:
pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=LlamaCppJSONGenerator(
        repo_id="microsoft/Phi-3-mini-4k-instruct-gguf",
        file_name="Phi-3-mini-4k-instruct-q4.gguf",
        schema_object=Order,
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: LlamaCppJSONGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [8]:
pipeline.run(
    {
        "prompt_builder": {"order": "Is it possible to have 12 margheritas?"},
        "llm": {"generation_kwargs": {"temperature": 0.5}},
    }
)

/Users/edoardoabati/projects/opensource/outlines-haystack/.venv/lib/python3.12/site-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `hf_hub_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
llama_model_load_from_file_impl: using device Metal (Apple M1 Pro) - 25553 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from /Users/edoardoabati/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct-gguf/snapshots/a64113399c2f6b8ad3e11c394733a2ddadaa7f33/./Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv  

{'llm': {'structured_replies': ['{"pizza": "Margherita", "number": 12']}}

## Choice Generation

In [9]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.llama_cpp import LlamaCppChoiceGenerator

In [10]:
generator = LlamaCppChoiceGenerator(
    repo_id="microsoft/Phi-3-mini-4k-instruct-gguf",
    file_name="Phi-3-mini-4k-instruct-q4.gguf",
    choices=["Positive", "Negative"],
)

### Standalone

In [11]:
generator.warm_up()

llama_model_load_from_file_impl: using device Metal (Apple M1 Pro) - 25553 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from /Users/edoardoabati/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct-gguf/snapshots/a64113399c2f6b8ad3e11c394733a2ddadaa7f33/./Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                     

In [12]:
generator.run(
    prompt="Classify the following statement: 'I love pizza'",
    generation_kwargs={"temperature": 0.5},
)

llama_perf_context_print:        load time =     434.20 ms
llama_perf_context_print: prompt eval time =     433.99 ms /    13 tokens (   33.38 ms per token,    29.95 tokens per second)
llama_perf_context_print:        eval time =      68.97 ms /     2 runs   (   34.48 ms per token,    29.00 tokens per second)
llama_perf_context_print:       total time =     503.85 ms /    15 tokens
llama_perf_context_print:    graphs reused =          1


{'choice': 'Positive'}

### In a Pipeline

In [13]:
prompt_template = "Classify the following statement: '{{statement}}'"

pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=LlamaCppChoiceGenerator(
        repo_id="microsoft/Phi-3-mini-4k-instruct-gguf",
        file_name="Phi-3-mini-4k-instruct-q4.gguf",
        choices=["Positive", "Negative"],
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: LlamaCppChoiceGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [14]:
pipeline.run(
    {
        "prompt_builder": {"statement": "I love Italian food"},
        "llm": {"generation_kwargs": {"temperature": 0.5}},
    }
)

llama_model_load_from_file_impl: using device Metal (Apple M1 Pro) - 25553 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from /Users/edoardoabati/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct-gguf/snapshots/a64113399c2f6b8ad3e11c394733a2ddadaa7f33/./Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                     

{'llm': {'choice': 'Positive'}}

## Text Generation

In [15]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.llama_cpp import LlamaCppTextGenerator

In [ ]:
generator = LlamaCppTextGenerator(
    repo_id="microsoft/Phi-3-mini-4k-instruct-gguf",
    file_name="Phi-3-mini-4k-instruct-q4.gguf",
)

### Standalone

In [17]:
generator.warm_up()

llama_model_load_from_file_impl: using device Metal (Apple M1 Pro) - 25553 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from /Users/edoardoabati/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct-gguf/snapshots/a64113399c2f6b8ad3e11c394733a2ddadaa7f33/./Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                     

In [21]:
generator.run(
    prompt="What is the capital of Italy?",
    generation_kwargs={"temperature": 0.5},
)

llama_perf_context_print:        load time =     334.41 ms
llama_perf_context_print: prompt eval time =     292.41 ms /     8 tokens (   36.55 ms per token,    27.36 tokens per second)
llama_perf_context_print:        eval time =     515.12 ms /    15 runs   (   34.34 ms per token,    29.12 tokens per second)
llama_perf_context_print:       total time =     809.62 ms /    23 tokens
llama_perf_context_print:    graphs reused =         15


{'replies': ["\n<|assistant|> The capital of Italy is Rome. It's a city rich in"]}

### In a Pipeline

In [19]:
prompt_template = "What is the capital of {{country}}?"

pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=LlamaCppTextGenerator(
        repo_id="microsoft/Phi-3-mini-4k-instruct-gguf",
        file_name="Phi-3-mini-4k-instruct-q4.gguf",
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: LlamaCppTextGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [22]:
pipeline.run(
    {
        "prompt_builder": {"country": "France"},
        "llm": {"generation_kwargs": {"temperature": 0.5}},
    }
)

llama_perf_context_print:        load time =     251.89 ms
llama_perf_context_print: prompt eval time =     509.20 ms /     8 tokens (   63.65 ms per token,    15.71 tokens per second)
llama_perf_context_print:        eval time =     516.20 ms /    15 runs   (   34.41 ms per token,    29.06 tokens per second)
llama_perf_context_print:       total time =    1026.97 ms /    23 tokens
llama_perf_context_print:    graphs reused =         15


{'llm': {'replies': ['\n<|assistant|> The capital of France is Paris.\n\nThis question is straightforward and']}}